In [18]:
# Load libraries
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
import math
import sys
from scipy.stats import spearmanr

# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
impIterative = IterativeImputer(missing_values=np.nan, sample_posterior=True, imputation_order='random')

from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt

In [19]:
url = "complete_dataset.csv"

In [21]:
original_dataset=read_csv(url, nrows=10_000)

C:\Users\ekech\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (9,10,11,20,21,22,25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [22]:
original_dataset

,CHILDID,PARENTID,S1_ID,S2_ID,S3_ID,S4_ID,S5_ST_ID,S5_ID,S6_ST_ID,S6_ID,...,C7CPTS81,C7CPTS82,C7CPTS83,C7CPTS84,C7CPTS85,C7CPTS86,C7CPTS87,C7CPTS88,C7CPTS89,C7CPTS90
0,0001001C,0001001P,1.0,1.0,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0001002C,0001002P,1.0,1.0,NaN,6005.0,6005.0,6005.0,6005.0,A053,...,2704.86,2681.98,2700.63,2697.53,2692.50,2665.01,2844.28,2665.86,2764.51,2720.46
2,0001003C,0001003P,1.0,1.0,NaN,9997.0,9997.0,9997.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0001004C,0001004P,1.0,1.0,NaN,6009.0,6009.0,6009.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0001005C,0001005P,1.0,1.0,NaN,3.0,7197.0,7197.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0685015C,0685015P,685.0,685.0,NaN,685.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,0685016C,0685016P,685.0,685.0,NaN,685.0,685.0,685.0,685.0,A355,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,0685017C,0685017P,685.0,685.0,NaN,685.0,685.0,685.0,685.0,0685,...,443.13,446.94,446.70,461.72,456.63,444.46,439.22,452.00,445.04,446.18
9998,0685018C,0685018P,685.0,685.0,NaN,685.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
dtypes = original_dataset.dtypes # Get the dtypes
cols = original_dataset.columns # Get the columns

In [26]:
dtype_dictionary = {} 
for c in cols:
    if str(dtypes[c]) == 'int64':
        dtype_dictionary[c] = 'float32' # Handle NANs in int columns
    else:
        dtype_dictionary[c] = str(dtypes[c])
    
    if ((c == 9) | (c == 10) | (c == 25) | (c == 26) | (c == 6145) | (c ==6146)):
        dtype_dictionary[c]  = str(dtypes[c])

In [27]:
full_dataset=read_csv(url, dtype=dtype_dictionary)

In [28]:
full_dataset

,CHILDID,PARENTID,S1_ID,S2_ID,S3_ID,S4_ID,S5_ST_ID,S5_ID,S6_ST_ID,S6_ID,...,C7CPTS81,C7CPTS82,C7CPTS83,C7CPTS84,C7CPTS85,C7CPTS86,C7CPTS87,C7CPTS88,C7CPTS89,C7CPTS90
0,0001001C,0001001P,1.0,1.0,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0001002C,0001002P,1.0,1.0,NaN,6005.0,6005.0,6005.0,6005.0,A053,...,2704.86,2681.98,2700.63,2697.53,2692.50,2665.01,2844.28,2665.86,2764.51,2720.46
2,0001003C,0001003P,1.0,1.0,NaN,9997.0,9997.0,9997.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0001004C,0001004P,1.0,1.0,NaN,6009.0,6009.0,6009.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0001005C,0001005P,1.0,1.0,NaN,3.0,7197.0,7197.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21404,3116024C,3116024P,3116.0,3116.0,NaN,3116.0,7294.0,7294.0,7294.0,3116,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21405,5213040C,5213040P,NaN,NaN,NaN,5213.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21406,5245040C,5245040P,NaN,NaN,NaN,5245.0,5245.0,5245.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21407,5245041C,5245041P,NaN,NaN,NaN,5245.0,5245.0,5245.0,5245.0,A290,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [29]:
full_dataset.to_pickle("./full_dataset.pkl")

In [ ]:
test = pd.read_pickle("./full_dataset.pkl")

In [15]:
dtypes = full_dataset.dtypes # Get the dtypes
cols = full_dataset.columns # Get the columns

In [25]:
sum(dtypes == 'object')

32

In [17]:
dtypes

CHILDID      object
PARENTID     object
S1_ID       float32
S2_ID       float64
S3_ID       float64
             ...   
C7CPTS86    float64
C7CPTS87    float64
C7CPTS88    float64
C7CPTS89    float64
C7CPTS90    float64
Length: 18928, dtype: object

## Determining the Best Y-Variable

In [10]:
# new_dataset = original_dataset.copy()
# new_dataset = new_dataset.replace(" ", np.NaN)
# new_dataset

,CHILDID,PARENTID,S1_ID,S2_ID,S3_ID,S4_ID,S5_ST_ID,S5_ID,S6_ST_ID,S6_ID,...,C7CPTS81,C7CPTS82,C7CPTS83,C7CPTS84,C7CPTS85,C7CPTS86,C7CPTS87,C7CPTS88,C7CPTS89,C7CPTS90
0,0001001C,0001001P,1,1.0,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0001002C,0001002P,1,1.0,NaN,6005.0,6005.0,6005.0,6005.0,A053,...,2704.86,2681.98,2700.63,2697.53,2692.50,2665.01,2844.28,2665.86,2764.51,2720.46
2,0001003C,0001003P,1,1.0,NaN,9997.0,9997.0,9997.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0001004C,0001004P,1,1.0,NaN,6009.0,6009.0,6009.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0001005C,0001005P,1,1.0,NaN,3.0,7197.0,7197.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0001006C,0001006P,1,1.0,NaN,6026.0,6026.0,6026.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0001007C,0001007P,1,1.0,NaN,6027.0,6027.0,6027.0,6027.0,6027,...,2595.60,2570.25,2588.66,2602.66,2524.02,2600.21,2705.82,2576.17,2631.86,2588.51
7,0001008C,0001008P,1,1.0,NaN,9997.0,9997.0,9997.0,9997.0,9997,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0001009C,0001009P,1,1.0,NaN,5569.0,5569.0,5569.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0001010C,0001010P,1,1.0,NaN,2.0,2.0,2.0,2.0,A053,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [11]:
# new_dataset.to_csv("complete_dataset.csv", encoding='utf-8', index=False)

### Remove the columns with the most NA values

In [12]:
def remove_highest_empties(df):
    half_of_df = len(df) / 2
    column_names = []
    
    for column_name, _ in df.iteritems():
        if (df[column_name].isna().sum() > half_of_df):
            column_names.append(column_name)
    
    df = df.drop(columns=column_names)
    return df

In [13]:
new_dataset = remove_highest_empties(new_dataset)
new_dataset

,CHILDID,PARENTID,S1_ID,S2_ID,S4_ID,S5_ST_ID,S5_ID,T1_ID,T2_ID,T4_ID,...,C1_5FP81,C1_5FP82,C1_5FP83,C1_5FP84,C1_5FP85,C1_5FP86,C1_5FP87,C1_5FP88,C1_5FP89,C1_5FP90
0,0001001C,0001001P,1,1.0,2.0,NaN,NaN,0001T01,0001T01,0002T06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0001002C,0001002P,1,1.0,6005.0,6005.0,6005.0,0001T01,0001T01,6005T01,...,352.02,351.46,350.73,353.71,347.83,348.89,365.14,349.65,355.08,345.66
2,0001003C,0001003P,1,1.0,9997.0,9997.0,9997.0,0001T01,0001T01,NaN,...,365.42,364.79,362.66,365.50,358.78,362.53,378.47,359.96,366.50,358.67
3,0001004C,0001004P,1,1.0,6009.0,6009.0,6009.0,0001T01,0001T01,6009T01,...,357.69,359.61,358.29,359.40,354.69,354.65,374.36,352.72,359.75,354.55
4,0001005C,0001005P,1,1.0,3.0,7197.0,7197.0,0001T01,0001T01,0003T01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0001006C,0001006P,1,1.0,6026.0,6026.0,6026.0,0001T01,0001T01,6026T99,...,357.69,359.61,358.29,359.40,354.69,354.65,374.36,352.72,359.75,354.55
6,0001007C,0001007P,1,1.0,6027.0,6027.0,6027.0,0001T01,0001T01,NaN,...,357.69,359.61,358.29,359.40,354.69,354.65,374.36,352.72,359.75,354.55
7,0001008C,0001008P,1,1.0,9997.0,9997.0,9997.0,0001T01,0001T01,NaN,...,473.02,470.46,472.52,478.36,463.26,467.29,491.20,473.92,479.30,467.21
8,0001009C,0001009P,1,1.0,5569.0,5569.0,5569.0,0001T01,0001T01,5569T01,...,442.35,442.37,441.47,442.48,438.17,439.75,459.23,445.50,444.63,430.25
9,0001010C,0001010P,1,1.0,2.0,2.0,2.0,0001T01,0001T01,0002T06,...,352.02,351.46,350.73,353.71,347.83,348.89,365.14,349.65,355.08,345.66


### Remove the NA values from the Y-Variable

##### Using 8th grade Math Scores as the Y-variable

In [14]:
clean_df = new_dataset.copy().dropna(subset=["C6R4MPF"])
clean_df

KeyError: ['C6R4MPF']

### Remove the variable relating to 5th Grade and Above

In [ ]:
def remove_unusable_variables(df):
    unusable_columns = []
    
    for column_name, _ in df.iteritems():
        if ('5' in column_name) or ('6' in column_name) or ('7' in column_name) or ('8' in column_name):
            if (column_name != 'C6R4MPF'):
                unusable_columns.append(column_name)
    
    df = df.drop(columns=unusable_columns)
    return df

In [ ]:
clean_df = remove_unusable_variables(clean_df)
clean_df

### Determining the Correlation Between Values

In [ ]:
# def find_correlation(df):
#     return df.corr()
# find the correlation between every column and the test column

In [ ]:
# corr_dict = find_correlation(clean_df)
# corr_dict

In [ ]:
# mask = (corr_dict['C6R4MPF'] > 0.3)
# mask

In [ ]:
# corr_dict['C6R4MPF'][mask]

In [ ]:
# corr_dict['C6R4MPF'].loc[((corr_dict['C6R4MPF'] > 0.3) | (corr_dict['C6R4MPF'] < -0.3))] # || < -0.8

In [ ]:
# mask.index.values

In [ ]:
# def find_covariance(df):
#     return df.cov()

In [ ]:
# cov_df = find_covariance(clean_df)
# cov_df

## Creating a new dataset with the columns we're using

In [ ]:
# dataset = original_dataset[["P4CMPEDU","P5CMPEDU","P6CMPEDU","C4R4MPF","C5R4MPF","W1INCCAT","W5INCCAT","W8INCCAT","WKMOMED","WKDADED","P1HMEMP","P1HDEMP","P7HMEMP","P7HDEMP","C6R4MPF"]]

In [ ]:
# dataset = clean_df.copy()

In [ ]:
# dataset.shape

In [ ]:
# Number of Non-NA values in each column
# dataset.count()

In [ ]:
# Number of NA values in each column
# dataset.isna().sum()

In [ ]:
# dataset.groupby('C6R4MPF').size()

In [ ]:
# cleandf=dataset.copy().dropna(subset=["C6R4MPF"])

In [ ]:
# cleandf.shape

In [ ]:
# cleandf.isna().sum()

In [ ]:
# print(cleandf.groupby('C6R4MPF').size())

In [ ]:
# dataset= cleandf[cleandf["C6R4MPF"]>"4"]
# print(dataset.groupby('C6R4MPF').size())

In [ ]:
# dataset.shape

In [ ]:
# Group some categories together
# conditions = [
#     (dataset['C6R4MPF'] >= "5.0") & (dataset['C6R4MPF'] < "7.0"),
#     (dataset['C6R4MPF'] >= "7.0")& (dataset['C6R4MPF'] <= "9.0")]
# choices = ['C - Low', 'A - High']
# scores = np.select(conditions, choices)
# dataset.loc[:, ('Score')] = scores
# dataset

In [ ]:
#remove the C6R4MPF column, now that we have the 'Score' category 
# dataset=dataset.drop('C6R4MPF',1)
# dataset.head()

In [ ]:
# print(dataset.groupby('Score').size())

In [ ]:
# dataset2=dataset[dataset['Score']=='A - High'].copy()

In [ ]:
# dataset2=dataset2.append(dataset[dataset['Score']=='C - Low'].sample(n=4777))

In [ ]:
# print(dataset2.groupby('Score').size())

In [ ]:
# print(dataset2.groupby('Score').size())

In [ ]:
# sample_size = math.floor(0.75*len(dataset2))
# sample_size

In [ ]:
# Creating our sample of random lines
# traindf=dataset2.sample(n=sample_size)
# #Listing what rows our sample is using
# traindf.index

In [ ]:
# Printing training dataset which imputation will be performed on
# traindf

In [ ]:
# Defining our validation data as rows in dataset2 that are not in our training data
# validationdf=dataset2.drop(traindf.index)
# len(validationdf)

In [ ]:
# taking out all NA's from validation dataset
# validationdf=validationdf.dropna()
# len(validationdf)

In [ ]:
# Do imputation on ALL columns of TRAIN 
# X = traindf[["P4CMPEDU","P5CMPEDU","P6CMPEDU","C4R4MPF","C5R4MPF","W1INCCAT","W5INCCAT","W8INCCAT","WKMOMED","WKDADED","P1HMEMP","P1HDEMP","P7HMEMP","P7HDEMP"]]

In [ ]:
# X

In [ ]:
# data_imputed=pd.DataFrame(impIterative.fit_transform(X),columns=["P4CMPEDU","P5CMPEDU","P6CMPEDU","C4R4MPF","C5R4MPF","W1INCCAT","W5INCCAT","W8INCCAT","WKMOMED","WKDADED","P1HMEMP","P1HDEMP","P7HMEMP","P7HDEMP"])

In [ ]:
# data_imputed

In [ ]:
# len(data_imputed)

In [ ]:
#Create a new column in data_imputed that doesn't exist yet to tack on the y-variable
# data_imputed["Score"]=traindf['Score'].values

In [ ]:
# data_imputed

In [ ]:
# data_imputed.shape

In [ ]:
# print(data_imputed.groupby('Score').size())

In [ ]:
# Split-out train dataset
# array = traindf.values
# X_train = array[:,0:14]
# Y_train= array[:,14]
# X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1)

In [ ]:
# Spot Check Algorithms
# models = []
# models.append(('LR', LogisticRegression(max_iter=10000, multi_class='multinomial')))
# models.append(('LDA', LinearDiscriminantAnalysis()))
# models.append(('KNN', KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski', weights='distance')))
# models.append(('CART', DecisionTreeClassifier()))
# models.append(('NB', GaussianNB()))
# models.append(('SVM', SVC(gamma='scale')))
# models.append(('RF', RandomForestClassifier(criterion='gini', n_estimators=10000, n_jobs=2)))

In [ ]:
# # evaluate each model in turn
# results = []
# names = []
# for name, model in models:
# 	kfold = StratifiedKFold(n_splits=7, random_state=1, shuffle=True)
# 	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
# 	results.append(cv_results)
# 	names.append(name)
# 	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

In [ ]:
# #Set validation set
# X_validation=validationdf.iloc[:,0:14]
# Y_validation=validationdf.iloc[:,14]

In [ ]:
# # Make predictions on validation dataset with SVM
 
# model = SVC(gamma='scale')
# model.fit(X_train, Y_train)
# predictions = model.predict(X_validation)

In [ ]:
# # Evaluate predictions
# print(accuracy_score(Y_validation, predictions))
# plot_confusion_matrix(model.fit(X_train, Y_train),X_validation,Y_validation,cmap=plt.cm.Blues)
# print(classification_report(Y_validation, predictions))

In [ ]:
# # Make predictions on validation dataset with RF
 
# model2 = RandomForestClassifier(criterion='gini', n_estimators=10000, n_jobs=2)
# model2.fit(X_train, Y_train)
# predictions2 = model2.predict(X_validation)

In [ ]:
# # Evaluate predictions
# print(accuracy_score(Y_validation, predictions2))
# plot_confusion_matrix(model2.fit(X_train, Y_train),X_validation,Y_validation,cmap=plt.cm.Blues)
# print(classification_report(Y_validation, predictions2))

In [ ]:
# feature_importance = model2.feature_importances_

# sorted_idx = np.argsort(feature_importance)[::-1]
# print("Feature Importance:\n")
# for i in range(len(X.columns)):
#     idx = sorted_idx[i]
#     print(f'{X.columns[idx]:20} {feature_importance[idx]:.3f}')